In [6]:
from random import randint
DEBUG = True

class Ship:
    def __init__(self, length, tp=1, x=None, y=None):
        self._x = x                   #_x, _y - координаты корабля (целые значения в диапазоне [0; size), где size - размер игрового поля);
        self._y = y
        self._length = length         # длина корабля (число палуб: целое значение: 1, 2, 3 или 4)
        self._tp = tp                 # ориентация корабля (1 - горизонтальная; 2 - вертикальная)
        self._is_move = True          # возможно ли перемещение корабля
        self._cells = [1] * length    # список длиной length, состоящий из единиц. при length=3, _cells = [1, 1, 1]
        
    def set_start_coords(self, x, y):
        '''
            установка начальных координат (запись значений в локальные атрибуты _x, _y)
        '''
        self._x, self._y = x, y
    
    def get_start_coords(self):
        '''
            получение начальных координат корабля в виде кортежа x, y
        '''
        return self._x, self._y

    def get_coords(self):
        '''
            получение координат корабля x1, y1, x2, y2
        '''
        x, y = self.get_start_coords()
        x1 = x2 = x 
        y1 = y2 = y
        if self._tp == 1:
            x2 += (self._length - 1)
        else:
            y2 += (self._length - 1)
        return x1, y1, x2, y2
    
    def get_mask(self):
        '''
            получение координат маски корабля x1, y1, x2, y2
        '''
        x1, y1, x2, y2 = self.get_coords()
        return x1-1, y1-1, x2+1, y2+1
    
    def move(self, go):
        '''
            перемещение корабля в направлении его ориентации на go клеток 
            (go = 1 - движение в одну сторону на клетку; 
            go = -1 - движение в другую сторону на одну клетку); 
            движение возможно только если флаг _is_move = True;
        '''
        if not self._is_move:
            if DEBUG:
                raise TypeError('Запрещено перемещение корабля') 
            return 
        go_to_x, go_to_y = self.get_start_coords()
        if self._tp == 1:
            go_to_x += go
        elif self._tp == 2:
            go_to_y += go
        return go_to_x, go_to_y
        
    def is_collide(self, ship):
        '''
            проверка на столкновение с другим кораблем ship (столкновением считается, 
            если другой корабль или пересекается с текущим или просто соприкасается, 
            в том числе и по диагонали); метод возвращает True, если столкновение есть и 
            False - в противном случае;
        '''
        x1A, y1A, x2A, y2A = self.get_coords()
        x1B, y1B, x2B, y2B = ship.get_mask()
        
        self_ship = set((x1, y1) for x1 in range(x1A, x2A+1) for y1 in range(y1A, y2A+1))
        other_ship_mask = set((x2, y2) for x2 in range(x1B, x2B+1) for y2 in range(y1B, y2B+1))
        
        if len(self_ship & other_ship_mask):
            return True
        return False 
        
    def is_out_pole(self, size):
        '''
            проверка на выход корабля за пределы игрового поля (size - размер игрового 
            поля, обычно, size = 10); возвращается булево значение True, 
            если корабль вышел из игрового поля и False - в противном случае;
        '''
        x1, y1, x2, y2 = self.get_coords()
        return x2 > size or y2 > size or x1 < 1 or y1 < 1
        
    def __getitem__(self, item):
        return self._cells[item]
    
    def __setitem__(self, item, value):
        self._cells[item] = value
        
    def __repr__(self):
        return self[:]
    
    
class GamePole:
    '''
        класс GamePole должен обеспечивать работу с игровым полем
    '''
    def __init__(self, size=10):
        self._size = size   # размеры игрового поля (обычно, size = 10)
        self._ships = []    # список из кораблей (объектов класса Ship)
        self.M = 4          # максимально палубный корабль
        self._sea = []      # Игровое поле
    
    def create_flotilla(self, deck):
        for i in range(1, deck + 1):
            for _ in range(i):
                self._ships.append(Ship(deck - i + 1, tp=randint(1, 2)))
    
    def placement_ships(self):
        pass
    
    def check_ship(self, ship):
        res = []
        assert ship.is_out_pole(self._size), 'выход корабля за пределы игрового поля'
        for check_ship in filter(lambda x: all(x for x in x.get_start_coords()), self.get_ships()[:]):
            if not ship is check_ship:
                assert ship.is_collide(check_ship), 'корабли пересекаются'
       
                    
    
    def init(self):
        '''
            начальная инициализация игрового поля; 
            здесь создается список из кораблей (объектов класса Ship)
            После этого, выполняется их расстановка на игровом поле со случайными 
            координатами так, чтобы корабли не пересекались между собой.
        '''
        self.clear_pole()
        self.create_flotilla(self.M)
        #self.placement_ships()
        self.show()               
        
        
        
    
    def get_ships(self):
        '''
            возвращает коллекцию _ships
        '''
        return self._ships
    
    
    def move_ships(self):
        '''
            перемещает каждый корабль из коллекции _ships на одну клетку 
            (случайным образом вперед или назад) в направлении ориентации корабля; 
            если перемещение в выбранную сторону невозможно (другой корабль 
            или пределы игрового поля), то попытаться переместиться в противоположную 
            сторону, иначе (если перемещения невозможны), оставаться на месте;
        '''
        pass
    
    def show(self):
        '''
            отображение игрового поля в консоли 
        '''
        for row in self.get_pole():
            print(*row)
       
    
    def __add__(self, ship):
        x1, y1, x2, y2 = ship.get_coords()
        points = set((x, y) for x in range(x1, x2+1) for y in range(y1, y2+1))
        for point, cell  in zip(points, ship[:]):
            x, y = point
            self._sea[y-1][x-1] = cell
            

    def get_pole(self):
        '''
            создание, если нет, и получение текущего игрового поля в виде двумерного 
            (вложенного) кортежа размерами size x size элементов.
        '''
        self._sea = [[0 for _ in range(self._size)] for _ in range(self._size)]
        for ship in self._ships:
            if all(bool(x) for x in ship.get_start_coords()):
                self + ship
        return self._sea
    
    def clear_pole(self):
        '''
            очистка игрового поля
        '''
        self._sea = []
        self._ships = []

class SeaBattle:
    '''
        для управления игровым процессом в целом
    '''
    pass

pole = GamePole()
pole.init()

0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0


In [14]:
pole._ships[0].set_start_coords(2,3)
pole._ships[1].set_start_coords(1,6)
pole._ships[2].set_start_coords(5,5)
pole._ships[3].set_start_coords(0,6)
pole._ships[4].set_start_coords(8,2)
pole.show()

0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0
0 1 0 0 0 0 0 1 0 0
0 1 0 0 0 0 0 0 0 0
0 1 0 0 1 0 0 0 0 0
1 1 1 0 1 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0


In [15]:
for ship in pole.get_ships():
    print(ship.get_start_coords())

(2, 3)
(1, 6)
(5, 5)
(0, 6)
(8, 2)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)


In [ ]:
[[0 for _ in range(10)] for _ in range(10)]

In [ ]:
a = [1,8,15]
gen = (x for x in a)#  if count(x) == 0
#print(list(gen))
a = [2,8,22]
print(list(gen))